<a href="https://colab.research.google.com/github/JekugaBrandon/Recommendation_system101/blob/master/recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [3]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

READING THE DATA **AVAILABLE**

In [5]:
ratings = tfds.load('movielens/100k-ratings', split="train")
#all available features of movies

movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/incomplete.FE182H_0.1.1/movielens-train.tfrecord*...…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [6]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))


movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

In [7]:
class MovieLensModel(tfrs.Model):
  #We derive from a custom base class to help reduce boilerplate
  # these are still plain keras Models.


  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
   super().__init__()

   # Setting up user and movie representations.
   self.user_model = user_model
   self.movie_model = movie_model

   #Set up retrieval task.

   self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:


    # Defining how the loss will be computed

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)



In [8]:
# definig the redrival task

In [17]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

#defining objectives
# Define the task without the FactorizedTopK metric for now
task = tfrs.tasks.Retrieval()

In [18]:
model = MovieLensModel(user_model, movie_model, task)

Creating the model, train it, and generate **predictions**

In [22]:
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))


# Training 3 epochs.


model.fit(ratings.batch(4096), epochs=3)

# Manual recommendation retrieval
user_id = "42"
user_embedding = model.user_model(tf.constant([user_id], dtype=tf.string))

# Get embeddings for all movies
all_movie_titles = movies.batch(1000).map(lambda x: x).as_numpy_iterator()
all_movie_titles = np.concatenate(list(all_movie_titles))
all_movie_titles_tensor = tf.constant(all_movie_titles, dtype=tf.string)
movie_embeddings = model.movie_model(all_movie_titles_tensor)

# Calculate scores (dot product)
scores = tf.linalg.matmul(user_embedding, movie_embeddings, transpose_b=True)

# Get top k movie indices
top_k = 3
top_k_indices = tf.argsort(scores, axis=-1, direction='DESCENDING')[0, :top_k]

# Get top k movie titles
top_k_titles = tf.gather(all_movie_titles_tensor, top_k_indices)

print(f"Top {top_k} recommendations for user {user_id}: {top_k_titles.numpy().tolist()}")

Epoch 1/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 13s 445ms/step - loss: 41873.9492 - regularization_loss: 0.0000e+00 - total_loss: 41873.9492
Epoch 2/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 21s 451ms/step - loss: 30827.4785 - regularization_loss: 0.0000e+00 - total_loss: 30827.4785
Epoch 3/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 414ms/step - loss: 30172.5742 - regularization_loss: 0.0000e+00 - total_loss: 30172.5742
Top 3 recommendations for user 42: [b'Just Cause (1995)', b'Rent-a-Kid (1995)', b'Two if by Sea (1996)']
